# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

Для того чтобы открыть файл в формате csv, импортируем нужную библиотеку. Для работы с табличными данными лучше всего подойдет Pandas.

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('')

Чтобы получить общее представление о датасете, ознакомимся с первыми и последними строками таблицы, а также изучим общую инфо о файле.

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
df.columns #проверка на наличие лишних пробелов в названии столбцов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод** 

Открыв файл с данными, мы получили таблицу с информацией о 21525 потенциальных заемщиках банка, которая включает в себя 12 столбцов. Условно все столбцы можно поделить на 2 группы: семья (кол-во детей, семейное положение) и карьера (трудовой стаж, доход). Для целей анализа необходимо будет оценить зависимость первой группы критериев на своевременное погашение долговых обязательств. 

Рассмотрев первые и последние 10 строк таблицы, а также общую информацию о ней, можно заметить ряд проблем:
1) Строчные буквы сочетаются с прописными 

2) Встречаются дубликаты

3) Некорректный формат отображения данных в столбцах 'days_employed' и 'total_income'

3) Отрицательные значения в столбце 'days_employed'

4) В данных есть пропущенные значения

## Предобработка данных

### Обработка пропусков

Для начала проверим, какой объем занимают пропущенные значения

In [7]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Можно заметить некоторую взаимосвязь между пропусками в столбцах 'days_employed' и 'total_income'. Возможные причины: ошибки при выгрузке данных или же часть клиентов действительно не работает, следовательно, и совокупный доход будет пустым.

Для удобства анализа и исключения ошибок необходимо избавиться от пустых значений в этих столбцах. Заменим их на 0. 

In [8]:
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Таким образом, мы избавились от пустых значений в данных.

Однако помимо пропусков в значениях существует ряд стилистических проблем. Разберем каждый столбец по отдельности.

In [9]:
# проверка столбца children
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце с кол-вом детей присутствуют отрицательные значения, а также несоразмерно большие. В первом случае подобное обозначение может предполагать отсутствие детей, тогда значения -1 можно заменить на 0. Во втором случае возможна опечатка: вместо 20 детей предполагается 2. Произведем замены. 

In [10]:
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 0)
df['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [11]:
# проверка столбца days_employed
df['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

При просмотре уникальных значений трудового стажа мы наблюдаем отрицательные значения. Рассмотрим отдельно положительные и отрицательные значения данного столбца.

In [12]:
df[df['days_employed'] > 0]['days_employed'].describe() #описание данных для положительных значений

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

In [13]:
df[df['days_employed'] < 0]['days_employed'].describe() #описание данных для отрицательных значений

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

В столбце с кол-вом дней занятости присутствуют отрицательные значения, что не характерно для данного параметра. Избавимся от '-'

In [14]:
df['days_employed'] = df['days_employed'].apply(abs)

От отрицательных значений мы избавились, но необходимо проверить, нет ли неоправданно больших значений.

In [15]:
df['days_employed'].sort_values(ascending = False).head(20)

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
4697     401635.032697
13420    401619.633298
17823    401614.475622
10991    401591.828457
8369     401590.452231
1184     401575.196728
4949     401573.905288
15192    401556.753550
5716     401524.259153
10484    401517.276388
16237    401486.706746
15599    401458.877781
701      401446.446720
14356    401440.834336
5762     401381.736156
Name: days_employed, dtype: float64

Если перевести максимальное значение в годы, получится 1100 лет. Следовательно, часть данных сохранена в другом формате. Можно предположить, что данные были выгружаны в часах, а не в днях. Переведем максимальные значения из часов в дни. 

In [16]:
df.loc[df['days_employed'] > 25000, 'days_employed'] = df.loc[df['days_employed'] > 25000, 'days_employed'] / 24
df['days_employed'].sort_values(ascending = False).head(20)

16335    18388.949901
4299     17615.563266
6954     16739.808353
10006    16738.158823
7664     16736.462226
2156     16736.436110
7794     16735.993752
4697     16734.793029
13420    16734.151387
17823    16733.936484
10991    16732.992852
8369     16732.935510
1184     16732.299864
4949     16732.246054
15192    16731.531398
5716     16730.177465
10484    16729.886516
16237    16728.612781
15599    16727.453241
701      16726.935280
Name: days_employed, dtype: float64

50 лет - уже похоже на правду!

In [17]:
df['dob_years'].sort_values(ascending = False) #с этим столбцом все нормально c оформлением

8880     75
11532    74
19642    74
4895     74
3460     74
         ..
4147      0
21179     0
7034      0
16042     0
6859      0
Name: dob_years, Length: 21525, dtype: int64

In [18]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Избавимся от строчной записи.

In [19]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [20]:
df['education_id'].value_counts() #с этим столбцом все нормально c оформлением

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [21]:
df['family_status'].unique() #аналогично образованию, избавимся от строчных букв

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [22]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [23]:
df['family_status_id'].value_counts() #с этим столбцом все нормально c оформлением

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [24]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Поскольку с неопределенным признаком пола всего 1 строка, можно заменить ее значение на наиболее распространенное - F.

In [25]:
df['gender'] = df['gender'].replace('XNA', 'F')
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [26]:
df['income_type'].value_counts() #с этим столбцом все нормально c оформлением

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [27]:
df['debt'].value_counts() #с этим столбцом все нормально c оформлением

0    19784
1     1741
Name: debt, dtype: int64

In [28]:
df['total_income'].sort_values(ascending = False) #с этим столбцом все нормально c оформлением

12412    2.265604e+06
19606    2.200852e+06
9169     1.726276e+06
20809    1.715018e+06
17178    1.711309e+06
             ...     
12407    0.000000e+00
12427    0.000000e+00
12436    0.000000e+00
12451    0.000000e+00
10762    0.000000e+00
Name: total_income, Length: 21525, dtype: float64

In [29]:
df['purpose'].unique() #с этим столбцом все нормально c оформлением

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

Были обнаружены все пропуски в таблице и заменены на подходящие значения. Также была проведена работа по общей стилизации и оформлению табличных данных.

### Замена типа данных

В датасете есть несколько столбцов, где уместен был бы другой тип данных. Сделаем заменту типов данных для данных столбцов.

Во-первых, столбец с занятостью в днях (days_employed): кол-во дней всегда целое число.

In [30]:
df['days_employed'] = df['days_employed'].astype('int')
df['days_employed'].head(10)

0     8437
1     4024
2     5623
3     4124
4    14177
5      926
6     2879
7      152
8     6929
9     2188
Name: days_employed, dtype: int64

Во-вторых, столбец с доходом (total_income): для лучшего восприятия можно округлить этот показатель.

In [31]:
df['total_income'] = df['total_income'].astype('int')
df['total_income'].head(10)

0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Теперь все данные оформлены корректно и имеют соответствующий тип данных.

### Обработка дубликатов

Для начала проверим, какие строки попадают в категорию полных дубликатов.

In [33]:
df_duplicated = df[df.duplicated(keep = False)].sort_values(by = list(df.columns))
df_duplicated

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,0,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,0,сделка с подержанным автомобилем
19321,0,0,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,0,сделка с подержанным автомобилем
3452,0,0,29,высшее,0,женат / замужем,0,M,сотрудник,0,0,покупка жилой недвижимости
18328,0,0,29,высшее,0,женат / замужем,0,M,сотрудник,0,0,покупка жилой недвижимости
4216,0,0,30,среднее,1,женат / замужем,0,M,сотрудник,0,0,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,0,34,среднее,1,женат / замужем,0,F,сотрудник,0,0,покупка жилья для сдачи
9013,2,0,36,высшее,0,женат / замужем,0,F,госслужащий,0,0,получение образования
14432,2,0,36,высшее,0,женат / замужем,0,F,госслужащий,0,0,получение образования
11033,2,0,39,среднее,1,гражданский брак,1,F,сотрудник,0,0,сыграть свадьбу


In [34]:
df = df.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Теперь данные готовы для анализа и неточности в оформлении, дублированные значения не повлияют на результаты исследования.

### Лемматизация

Все данные можно поделить на группы (категории) по столбцу целей кредита ('purpose'). Для этого необходимо определить основу в каждой ячейке столбца. Нам поможет лемматизация. 

In [35]:
m = Mystem()

In [36]:
purpose_unique = ' '.join(df['purpose'].unique()) #находим уникальные значения столбца и объединяем их в строку
lemmas = m.lemmatize(purpose_unique) #находим основу уникальных значений

Определим частоту упоминания каждой основы, чтобы на ее основе определить будущие категории.

In [37]:
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Создадим функцию, которая определит категорию каждому клиенту банка исходя из целей кредитования.

In [38]:
purpose = ['жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба']
def categorize(value):
    lemmas = m.lemmatize(value)
    for item in purpose:
        if item in lemmas:
            return item
    return 'прочее'

In [39]:
df['purpose_category'] = df['purpose'].apply(categorize) #добавим новый столбец в таблицу с полученной категорией 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Таким образом, мы получили новый столбец с категорией.

### Категоризация данных

В настоящий момент наш датасет наполнен большим объемом информации. Чтобы было проще ориентироваться в данных, можно создать "словари".

In [40]:
df_dict_education = df[['education_id', 'education']].drop_duplicates().set_index('education_id')
df_dict_education

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


In [41]:
df_dict_education = df[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')
df_dict_education

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,не женат / не замужем


Можно создать категории в зависимости от совокупного дохода клиента банка.

In [42]:
def income_category(income): #создаем функцию для определения уровня дохода
    if income  < 50000:
        return 'низкий'
    if income  > 150000:
        return 'высокий'
    return 'средний'
df['income_category'] = df['total_income'].apply(income_category) #добавляем новые сведения в таблицу
df_dict_income = df[['income_category']].drop_duplicates().set_index('income_category')
df_dict_income['income_description'] = ['более 150 000', '50 000 - 150 000', 'менее 50 000']
df_dict_income

,income_description
income_category,
высокий,более 150 000
средний,50 000 - 150 000
низкий,менее 50 000


**Вывод**

Были созданы небольшие "словари", помогающие определить значения id

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
df_children_debt = df.pivot_table(index = 'children', columns = 'debt', values = 'total_income', aggfunc = 'count')
df_children_debt.columns = ['no debt', 'debt']
df_children_debt['ratio'] = df_children_debt['debt'] / (df_children_debt['no debt'] + df_children_debt['debt'])
df_children_debt

,no debt,debt,ratio
children,,,
0,13142.0,1072.0,0.075419
1,4364.0,444.0,0.092346
2,1858.0,194.0,0.094542
3,303.0,27.0,0.081818
4,37.0,4.0,0.097561
5,9.0,NaN,NaN


**Вывод**
Разница между клиентами, у которых есть дети, и бездетными невелика. Но все же должников с детьми больше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
df_family_debt = df.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = 'count')
df_family_debt.columns = ['no debt', 'debt']
df_family_debt['ratio'] = df_family_debt['debt'] / (df_family_debt['no debt'] + df_family_debt['debt'])
df_family_debt

,no debt,debt,ratio
family_status,,,
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452
не женат / не замужем,2536,274,0.097509


**Вывод**
Взаимосвязь между семейным положением и возвратом кредита имеется. Ниже всего процент у разведенных и вдовцов/вдов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
df_income_debt = df.pivot_table(index = 'income_category', columns = 'debt', values = 'total_income', aggfunc = 'count')
df_income_debt.columns = ['no debt', 'debt']
df_income_debt['ratio'] = df_income_debt['debt'] / (df_income_debt['no debt'] + df_income_debt['debt'])
df_income_debt

,no debt,debt,ratio
income_category,,,
высокий,8458,726,0.079051
низкий,2282,193,0.077980
средний,8973,822,0.083920


**Вывод**
Зависимость между уровнем дохода и возвратом кредита есть. В категории высокого дохода процент должников ниже.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
df_purpose_debt = df.pivot_table(index = 'purpose_category', columns = 'debt', values = 'total_income', aggfunc = 'count')
df_purpose_debt.columns = ['no debt', 'debt']
df_purpose_debt['ratio'] = df_purpose_debt['debt'] / (df_purpose_debt['no debt'] + df_purpose_debt['debt'])
df_purpose_debt

,no debt,debt,ratio
purpose_category,,,
автомобиль,3903,403,0.093590
жилье,4152,308,0.069058
недвижимость,5877,474,0.074634
образование,3643,370,0.092200
свадьба,2138,186,0.080034


**Вывод** 
В зависимости от целей кредита, вероятность возврата меняется. Минимальный процент задолженности приходится на недвижимость. 

## Общий вывод

Подводя итог, можно сказать что <b>наличие/отсутствие детей, семейное положение, уровень дохода или же цель кредитования оказывает влияние на факт погашения кредита в срок</b>. 

При принятии решения необходимо это учитывать, ведь даже незначительные различия по возврату кредитов (до 2%) в реальных условиях могут вызвать увеличение кредитного портфеля банка и, как следствие, повысит затраты банка на формирование обязательных резервов.

Ниже всего процент невозврата у <b>разведенных и вдовцов/вдов</b>, а также <b>клиентов без детей</b>.

Наиболее обеспеченные типы кредитования - на покупку <b>недвижимости и жилья</b> (процент должников по этим целям мельше всего).

По уровню заработка особое внимание стоит уделять клиентам с заработком <b>50 000 - 150 000 руб.</b> (<b>процент должников выше</b> всего именно по ним).

Чтобы в будущем не сталкиваться с проблемой некорректного отображения данных, можно предложить <b>ряд рекомендаций</b>:

- считать занятость не в днях, а в годах (преобладающее кол-во людей не знают точный стаж в днях, зато с годами не возникнет проблем);
- привести выгрузку данных к единой форме, чтобы исключить разночтение;
- ограничить заведение аномальных данных, которые могут повлиять на результаты анализа (например, число детей -1).

